### Práctica 8
### Tratamiento de datos masivos

Jorge Sánchez-Barriga Pons


Partimos del fichero p8data.csv que tiene los datos obtenidos a partir de los sensores del móvil en el bolsillo de personas que han hecho distintos ejercicios, además de algunos datos personales.

*0 -> subir escaleras

*1 -> bajarlas

*2 -> sentarse

*3 -> ponerse de pie

*4 -> andar 

*5 -> correr

El objetivo de esta práctica es intentar predecir el tipo de movimiento. Para ello se pide elaborar un modelo clasificador que sea capaz de predecir la etiqueta (el tipo de movimiento). 

Se juzgará el resultado atendiendo al menor valor F-score obtenido de los F-scores asociados a cada categoría.

![imagen.png](https://wikimedia.org/api/rest_v1/media/math/render/svg/057ffc6b4fa80dc1c0e1f2f1f6b598c38cdd7c23)

In [1]:
%matplotlib inline
import os
# cambiamos las variables del sistema
spark = 'C:\\hlocal\\tdm\\spark\\hadoop\\spark-2.3.2-bin-hadoop2.7'
# en el path se añade
path = os.environ.get('PATH') 
path = path+ ';'+spark+'\\bin;'
os.environ['PATH'] = path
os.environ['SPARK_HOME']= spark 
os.environ['HADOOP_HOME']= spark 
os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

# si da problema con collect quizás haya que poner java_home
#os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
labs = 'C:\\JDK\\jdk8-64bits'
os.environ['JAVA_HOME']= labs
os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+path


import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
print("Preparado!!")


Preparado!!


In [17]:
# Solución
path = 'c:/hlocal/tdm/p8data.csv'
raw = spark.read.format("com.databricks.spark.csv")\
            .options(header='true', inferschema='true') \
            .load(path)

raw.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
label,1059781,2.691240926191355,1.433020152339594,0,5
weight,1059781,72.12058340355225,15.981282468886219,48.0,102.0
height,1059781,173.9890043320271,8.798617832930004,161.0,190.0
age,1059781,28.83205681173752,5.303467347114264,18.0,46.0
gender,1059781,0.5736487066667547,0.494546356559385,0.0,1.0
attitude_roll,1059781,2990.120506972667,2593.5703717606298,0,16422
attitude_pitch,1059781,-0.17603031809685282,1.566705346795994,-3.1415900000000003,3.141578
attitude_yaw,1059781,-0.9866611563643813,0.4940793460012992,-1.569648,1.566183
gravity_x,1059781,-0.14935694859692678,1.5448005520063275,-3.141588,3.141584


In [29]:
from pyspark.ml.feature import VectorAssembler
input_cols = [col for col in raw.columns if col not in ['label']]
assembler = VectorAssembler(
        inputCols=input_cols,
        outputCol='features')

datosprep =  assembler.transform(raw)

datosprep.show(1)


+-----+------+------+----+------+-------------+--------------+------------+---------+---------+---------+--------------------+--------------+--------------+------------------+------------------+--------------------+--------------------+
|label|weight|height| age|gender|attitude_roll|attitude_pitch|attitude_yaw|gravity_x|gravity_y|gravity_z|      rotationRate_x|rotationRate_y|rotationRate_z|userAcceleration_x|userAcceleration_y|  userAcceleration_z|            features|
+-----+------+------+----+------+-------------+--------------+------------+---------+---------+---------+--------------------+--------------+--------------+------------------+------------------+--------------------+--------------------+
|    0| 102.0| 188.0|46.0|   1.0|            0|      1.528132|   -0.733896| 0.696372| 0.741895| 0.669768|-0.03167200000000...|      0.316738|       0.77818|1.0827639999999998|          0.294894|-0.18449300000000002|[102.0,188.0,46.0...|
+-----+------+------+----+------+-------------+-----

In [30]:
splits = datosprep.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]


In [62]:
from pyspark.ml.classification import RandomForestClassifier

regresor = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees=20, maxDepth=10)

model = regresor.fit(train)
res_test = model.transform(test)


In [63]:
res_test.show(5)

+-----+------+------+----+------+-------------+--------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|weight|height| age|gender|attitude_roll|attitude_pitch|       attitude_yaw|         gravity_x|          gravity_y|         gravity_z|      rotationRate_x|     rotationRate_y|     rotationRate_z|  userAcceleration_x| userAcceleration_y|  userAcceleration_z|            features|       rawPrediction|         probability|prediction|
+-----+------+------+----+------+-------------+--------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------

In [64]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(res_test)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.186667


In [ ]:
# esto para grabar 
#path = 'c:/hlocal/tdm/modelo'
path = 'c:/hlocal/tdm/modelo'
model.write.save(path)